
# Mapping Opportunity & Designing Place‑Based Mobility Policy — Assignment Template (All Plotly)

**Author:** Zachary A. Smith, Ph.D.  
**Notebook purpose:** Aligns with Tasks A–E of the assignment. Uses Opportunity Atlas Module‑1 tract outcomes and Table‑9 covariates (2010 Census tract geographies). All charts in **Plotly**.

> **Read first**
> - Opportunity Atlas overview/data & methods (Module 1).  
> - Table‑9 covariates codebook (variable names such as `rent_twobed2015`, `jobs_total_5mi_2015`, `frac_coll_plus2010`).  
> - TIGER/Line tract and place layers (2010 tracts; Newark place FIPS 36‑49891).  
> - Ethics in preprocessing (small‑cell privacy, suppression, transparency).

**Key reminders**
- OA Module‑1 tract outcomes are on **2010 Census tracts**. Keep vintages consistent when mapping, or disclose when using newer TIGER layers for **visual overlays only**.  
- When summarizing people, weight by the appropriate **count** column (e.g., `count_pooled`, `count_black`).  
- Consider **small‑cell reliability**: set a minimum subgroup size (e.g., `count_black ≥ 50`) and document your rationale in the ethics log.


In [ ]:

# %% Setup
import os, json, math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

pd.options.display.float_format = '{:,.2f}'.format

TEMPLATE = 'plotly_white'
COLOR_SEQ = px.colors.qualitative.D3

def add_layout(fig, title, x=None, y=None):
    fig.update_layout(template=TEMPLATE, title=title, margin=dict(l=10,r=10,t=60,b=10),
                      legend=dict(orientation='h', y=1.08))
    if x: fig.update_xaxes(title=x)
    if y: fig.update_yaxes(title=y)
    return fig

def zp(n, width):
    return str(int(n)).zfill(width)

def make_geoid10(state, county, tract):
    return zp(state,2)+zp(county,3)+zp(tract,6)

print('Environment ready.')



## A. Select & Frame the Region

Fill these with your chosen region codes (examples shown for Wayne County, NY; Newark place FIPS **49891**):


In [ ]:

STATE_FIPS = 36       # New York
COUNTY_FIPS = 117     # Wayne County
PLACE_FIPS  = '49891' # Newark (for optional PLACE overlay)
print(STATE_FIPS, COUNTY_FIPS, PLACE_FIPS)



## B. Acquire, Clean, and Merge (Module‑1 outcomes + Table‑9 covariates)

Students: paste/download the **public CSVs** for Module‑1 tract outcomes and **Table‑9** covariates, confirm variable names against the codebook, then merge on **`geoid10`** (2010 tracts).

> If you also have `atlas.dta`, you may use it as a cross‑check, but the rubric expects you to demonstrate the **CSV + codebook** route.


In [ ]:

# === Provide paths (local or remote). Keep dtype Int for state/county/tract, then build geoid10 ===
OA_OUTCOMES_CSV = "path/to/oa_module1_outcomes_tract.csv"     # <-- EDIT
OA_TABLE9_CSV   = "path/to/oa_table9_covariates_tract.csv"    # <-- EDIT

# Load outcomes & covariates
outcomes = pd.read_csv(OA_OUTCOMES_CSV, dtype={"state":"Int64","county":"Int64","tract":"Int64"})
covars   = pd.read_csv(OA_TABLE9_CSV,   dtype={"state":"Int64","county":"Int64","tract":"Int64"})

# Build geoid10 and subset to state
for df in (outcomes, covars):
    df["geoid10"] = df.apply(lambda r: make_geoid10(r["state"], r["county"], r["tract"]), axis=1)

# Merge outcomes+covariates
merged = outcomes.merge(
    covars.drop(columns=["state","county","tract"], errors='ignore'),
    on="geoid10", how="left"
)

# Keep only the chosen state; keep county for regional slicing
merged["state"]  = outcomes["state"]
merged["county"] = outcomes["county"]
merged["tract"]  = outcomes["tract"]

ny = merged[merged["state"]==STATE_FIPS].copy()
wayne = ny[ny["county"]==COUNTY_FIPS].copy()
print("Merged rows (NY):", len(ny), "| Wayne:", len(wayne))

# Log missingness in key covariates for transparency
key_covs = ["poor_share2010","singleparent_share2010","jobs_total_5mi_2015","rent_twobed2015","frac_coll_plus2010"]
missing_report = ny[key_covs].isna().sum().to_frame("n_missing")
missing_report



### Optional: Cross‑check with `atlas.dta`
If you maintain `atlas.dta`, you can load it to verify a few fields (e.g., `kfr_pooled_p25`), but 
**grading will focus on the CSV + codebook merge.**


In [ ]:

# Optional verification (falls back silently if file absent)
try:
    ATLAS_DTA = "atlas.dta"
    if os.path.exists(ATLAS_DTA):
        atlas = pd.read_stata(ATLAS_DTA)
        atlas["geoid10"] = atlas.apply(lambda r: make_geoid10(r["state"], r["county"], r["tract"]), axis=1)
        print("atlas.dta loaded for cross-check only.")
except Exception as e:
    print("atlas.dta check skipped:", e)



## C. Explore & Diagnose

**Weighted descriptives**: compute pooled (people‑weighted) summaries using `count_pooled`.  
**Correlations & OLS**: Use 2–3 Table‑9 covariates and interpret carefully.


In [ ]:

# Weighted pooled descriptive stats for outcome: kfr_pooled_p25

def wmean(s, w):
    ok = (~s.isna()) & (~w.isna())
    return float(np.average(s[ok], weights=w[ok])) if ok.sum() else float("nan")

ny_pooled = wmean(ny["kfr_pooled_p25"], ny["count_pooled"])
wayne_pooled = wmean(wayne["kfr_pooled_p25"], wayne["count_pooled"])

pd.DataFrame({
    "Geography":["NY statewide","Wayne County"],
    "Weighted mean kfr_pooled_p25":[ny_pooled, wayne_pooled]
})


In [ ]:

# Correlations and a compact OLS (robust SEs)
cols = ["kfr_pooled_p25","poor_share2010","rent_twobed2015","frac_coll_plus2010"]
wayne_corr = wayne[cols].dropna().corr()
wayne_corr


In [ ]:

import statsmodels.api as sm

reg_df = ny[["kfr_pooled_p25","poor_share2010","rent_twobed2015","frac_coll_plus2010"]].dropna().astype(float)
Y = reg_df["kfr_pooled_p25"]
X = sm.add_constant(reg_df[["poor_share2010","rent_twobed2015","frac_coll_plus2010"]])
ols = sm.OLS(Y, X).fit(cov_type="HC3")
print(ols.summary().tables[1])



## D. Define “Opportunity Zones” (two approaches)

### D1) Newark benchmark (race‑specific): Black @ P25 above Newark’s tract value (36117021200)
- Add a **minimum subgroup size** to avoid small‑cell noise (default `count_black ≥ 50`).

### D2) Rubric‑aligned multi‑criteria (pooled):
- **Top quartile** of `kfr_pooled_p25` **within** your region (Wayne or state),
- **Below median** `rent_twobed2015`, and
- **≥ median** `frac_coll_plus2010`.

Include **sensitivity checks** (e.g., use 40th/60th percentiles, or z‑score thresholds) and discuss trade‑offs.


In [ ]:

# --- D1: Newark benchmark (Black @ P25) ---
min_black_children = 50

# Identify Newark tract row in merged if available; otherwise build key directly
newark_geoid10 = make_geoid10(STATE_FIPS, COUNTY_FIPS, 21200)

# If Newark row exists, read from merged; else default to atlas if loaded
if newark_geoid10 in set(ny.get("geoid10", [])):
    newark_kfr_black_p25 = float(ny.loc[ny["geoid10"]==newark_geoid10, "kfr_black_p25"].dropna().iloc[0])
elif 'atlas' in globals():
    newark_kfr_black_p25 = float(atlas.loc[atlas.apply(lambda r: make_geoid10(r['state'], r['county'], r['tract']), axis=1)==newark_geoid10, "kfr_black_p25"].dropna().iloc[0])
else:
    newark_kfr_black_p25 = float('nan')
    print("Warning: Newark kfr_black_p25 not found in merged CSVs; set atlas.dta or enter manually.")

ny_black = ny[ny["count_black"] >= min_black_children].copy()
ny_black["delta_vs_newark"] = ny_black["kfr_black_p25"] - newark_kfr_black_p25
ny_black["is_opzone_benchmark"] = ny_black["delta_vs_newark"] > 0

print("Benchmark OZ count (NY, Black@P25, count_black ≥", min_black_children, "):", int(ny_black["is_opzone_benchmark"].sum()))
ny_black[["geoid10","kfr_black_p25","count_black","delta_vs_newark","is_opzone_benchmark"]].head()


In [ ]:

# --- D2: Multi-criteria OZ (Wayne County example) ---
county = wayne.copy()

kfr_q = county["kfr_pooled_p25"].quantile(0.75)
rent_med = county["rent_twobed2015"].median()
college_med = county["frac_coll_plus2010"].median()

county["oz_multi"] = (
    (county["kfr_pooled_p25"] >= kfr_q) &
    (county["rent_twobed2015"] <= rent_med) &
    (county["frac_coll_plus2010"] >= college_med)
)

print("Multi-criteria OZ count (Wayne):", int(county["oz_multi"].sum()))
county.loc[county["oz_multi"], ["geoid10","kfr_pooled_p25","rent_twobed2015","frac_coll_plus2010"]].head()


In [ ]:

# Sensitivity toggles (students can adjust)
alt_rent = county["rent_twobed2015"].quantile(0.40)
alt_col  = county["frac_coll_plus2010"].quantile(0.60)
county["oz_multi_sens"] = (
    (county["kfr_pooled_p25"] >= kfr_q) &
    (county["rent_twobed2015"] <= alt_rent) &
    (county["frac_coll_plus2010"] >= alt_col)
)
print("Multi-criteria (tighter) OZ count (Wayne, rent ≤ 40th, college ≥ 60th):", int(county["oz_multi_sens"].sum()))



## Mapping (Plotly choropleths)
- Convert TIGER/Line **2010** NY tracts to GeoJSON (done earlier or via provided conversion script).  
- Join on **`geoid10` ↔ `GEOID10`**.  
- Optional: overlay or clip by **Newark PLACE (36‑49891)** for a focused view (visual‑only if mixing vintages).


In [ ]:

# Load GeoJSON (produced earlier) and map delta vs Newark (Black@P25) + multi-criteria OZ
GEOJSON_PATH = "Opportunity_Atlas_Senior_Sem/geojson/tl_2010_36_tract10.geojson"  # <-- EDIT if needed

with open(GEOJSON_PATH, 'r') as f:
    ny_geo = json.load(f)

# Build a frame keyed by 2010 geoid
map_bench = ny_black[["geoid10","kfr_black_p25","count_black","delta_vs_newark","is_opzone_benchmark"]].copy()
map_bench = map_bench.rename(columns={"geoid10":"geoid"})

fig_map = px.choropleth_mapbox(
    map_bench, geojson=ny_geo, locations='geoid', featureidkey='properties.GEOID10',
    color='delta_vs_newark', color_continuous_scale=['#440154','#3b528b','#21908d','#5ec962','#fde725'],
    mapbox_style='open-street-map', opacity=0.75, zoom=5, center={'lat':42.95,'lon':-75.5},
    hover_data={'geoid':True,'kfr_black_p25':':,.0f','count_black':True,'delta_vs_newark':':,.0f','is_opzone_benchmark':True},
    title='Δ vs Newark (Black @ P25), New York Tracts — 2010 geographies'
)
add_layout(fig_map, fig_map.layout.title.text)
fig_map.show()

# Multi-criteria binary highlight
map_multi = county[["geoid10","oz_multi","kfr_pooled_p25","rent_twobed2015","frac_coll_plus2010"]].rename(columns={"geoid10":"geoid"})
fig_multi = px.choropleth_mapbox(
    map_multi.assign(Status=lambda d: np.where(d["oz_multi"], 'Selected OZ', 'Not selected')),
    geojson=ny_geo, locations='geoid', featureidkey='properties.GEOID10',
    color='Status', color_discrete_sequence=['#2ca02c','#d62728'],
    mapbox_style='open-street-map', opacity=0.80, zoom=7, center={'lat':43.05,'lon':-77.1},
    title='Opportunity Zones (Multi‑criteria) — Wayne County'
)
add_layout(fig_multi, fig_multi.layout.title.text)
fig_multi.show()



## E. Policy Memo (export & pointers)
Export a small table (GEOID10 + key metrics) for your memo and cite your sources.


In [ ]:

# Export shortlists for memo tables
county.loc[county["oz_multi"], ["geoid10","kfr_pooled_p25","rent_twobed2015","frac_coll_plus2010"]]       .to_csv('oz_multi_wayne_table.csv', index=False)

map_bench[map_bench["is_opzone_benchmark"]]     .to_csv('oz_benchmark_black_p25_ny.csv', index=False)

print('Saved: oz_multi_wayne_table.csv; oz_benchmark_black_p25_ny.csv')



## Decisions & Data Ethics Log (students: fill in)

- **Rows dropped (merge/analysis):** ___ (counts and reasons).  
- **Minimum subgroup thresholds:** e.g., `count_black ≥ 50` (rationale: small‑cell reliability & privacy).  
- **Transformations/filters:** e.g., winsorizing rent; NA handling.  
- **Geography vintage:** OA = 2010 tracts; PLACE (2024 TIGER) used for **overlay only** unless crosswalk applied.  
- **Limitations:** Observational estimates, subgroup sparsity, ecological inference cautions.
